In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler


Grupo A ShapiroResult(statistic=0.9780016294384767, pvalue=0.9492885624975839)
Grupo B ShapiroResult(statistic=0.9780016294384769, pvalue=0.9492885624975852)
